In [ ]:
import numpy as np

In [ ]:
def idx(iA, jB, n=3):
    return iA * n + jB

def normalize_adjacency(A):
    norm = np.linalg.norm(A, 'fro')
    if norm > 0:
        A = A / norm
    return A


def random_weighted_graph_qutrit(p=0.5):
    """
    Random weighted graph on 3x3 vertices with fixed tensor labeling
    and Frobenius normalization ||A||_F = 1
    """
    A = np.zeros((9, 9))
    for iA in range(3):
        for jB in range(3):
            u = idx(iA, jB)
            for kA in range(3):
                for lB in range(3):
                    v = idx(kA, lB)
                    if u < v and np.random.rand() < p:
                        w = np.random.rand()
                        A[u, v] = w
                        A[v, u] = w
    return normalize_adjacency(A)
def partial_transpose_adj(A, m=3, n=3):
    AT = np.zeros_like(A)
    for iA in range(m):
        for jB in range(n):
            u = idx(iA, jB)
            for kA in range(m):
                for lB in range(n):
                    v = idx(kA, lB)
                    AT[idx(iA, lB), idx(kA, jB)] = A[u, v]
    return AT

def alpha_positivity(A):
    d = A.sum(axis=1)
    delta = d.min()
    lambda_min = np.linalg.eigvalsh(A)[0]

    if lambda_min >= 0:
        return 0.0
    return lambda_min / (lambda_min - delta)

def alpha_ppt_threshold(A, m=3, n=3):
    d = A.sum(axis=1)
    dG = d.sum()
    sum_d2 = np.sum(d**2)

    term = (dG**2) / (m*n - 1) - sum_d2
    if term <= 0:
        return 1.0

    return 1.0 / (1.0 + np.sqrt(term / np.linalg.norm(A, 'fro')**2))

def triangle_term(A_TB):
    return np.trace(A_TB @ A_TB @ A_TB) / 6.0

def violates_moment_inequality(A, alpha):
    d = A.sum(axis=1)
    dG = d.sum()
    sum_d2 = np.sum(d**2)
    sum_d3 = np.sum(d**3)

    A_TB = partial_transpose_adj(A)

    edge_sum = 0.0
    for i in range(9):
        for j in range(i+1, 9):
            if A_TB[i, j] != 0:
                edge_sum += (d[i] + d[j]) * (A_TB[i, j]**2)

    tri = triangle_term(A_TB)
    beta = (1 - alpha) / alpha

    LHS = (sum_d2 + beta**2 * np.linalg.norm(A, 'fro')**2)**2
    RHS = dG * (
        sum_d3
        + 3 * beta**2 * edge_sum
        + 6 * beta**3 * tri
    )

    return LHS > RHS
def density_matrix_from_graph(A, alpha):
    d = A.sum(axis=1)
    dG = d.sum()
    if dG <= 0:
        return None
    D = np.diag(d)
    return (alpha * D + (1 - alpha) * A) / (alpha * dG)

In [ ]:
def classify_state_from_graph(A, alpha):
    alpha0 = alpha_positivity(A)
    if alpha < alpha0:
        return None  # unphysical

    alpha_ppt = alpha_ppt_threshold(A)

    if alpha < alpha_ppt:
        return "ENT"  # NPT entangled

    if violates_moment_inequality(A, alpha):
        return "BE"   # PPT entangled (bound)
    else:
        return "SEP"  # separable (paper-certified)

def stratified_alphas(alpha_low, alpha_high, k, eps_frac=0.1):
    """
    Returns k alpha values stratified into:
    - lower boundary
    - middle
    - upper boundary
    """
    if alpha_high <= alpha_low:
        return []

    width = alpha_high - alpha_low
    eps = eps_frac * width

    alphas = []

    # lower boundary
    alphas.append(
        np.random.uniform(alpha_low, alpha_low + eps)
    )

    # middle
    if k > 2:
        alphas.extend(
            np.random.uniform(
                alpha_low + eps,
                alpha_high - eps,
                k - 2
            )
        )

    # upper boundary
    alphas.append(
        np.random.uniform(alpha_high - eps, alpha_high)
    )

    return alphas

In [ ]:
def gell_mann_matrices():
    l = []
    l.append(np.array([[0,1,0],[1,0,0],[0,0,0]]))
    l.append(np.array([[0,-1j,0],[1j,0,0],[0,0,0]]))
    l.append(np.array([[1,0,0],[0,-1,0],[0,0,0]]))
    l.append(np.array([[0,0,1],[0,0,0],[1,0,0]]))
    l.append(np.array([[0,0,-1j],[0,0,0],[1j,0,0]]))
    l.append(np.array([[0,0,0],[0,0,1],[0,1,0]]))
    l.append(np.array([[0,0,0],[0,0,-1j],[0,1j,0]]))
    l.append((1/np.sqrt(3))*np.array([[1,0,0],[0,1,0],[0,0,-2]]))
    return l

I3 = np.eye(3)
GM = [I3] + gell_mann_matrices()

def gell_mann_features(rho):
    feats = []
    for i in range(9):
        for j in range(9):
            if i == 0 and j == 0:
                continue  # skip I ⊗ I
            op = np.kron(GM[i], GM[j])
            feats.append(np.trace(rho @ op).real)
    return np.array(feats)

In [ ]:
def random_unitary(n=3):
    """
    Haar-random unitary from U(n)
    """
    Z = (np.random.randn(n, n) + 1j * np.random.randn(n, n)) / np.sqrt(2)
    Q, R = np.linalg.qr(Z)

    # Fix phases
    d = np.diag(R)
    Q = Q * (d / np.abs(d))

    return Q

def apply_local_unitary(rho):
    UA = random_unitary(3)
    UB = random_unitary(3)
    U = np.kron(UA, UB)
    return U @ rho @ U.conj().T

In [ ]:
def build_dataset(
    n,
    p_min=0.1,
    p_max=0.9,
    k_ent=3,
    k_sep=3,
    k_lu=2,
    eps_frac=0.1,
    max_trials=10_000_000
):
    """
    Balanced dataset with:
      - randomized p
      - stratified alpha sampling
      - multiple alpha per graph
      - local unitary augmentation
    """

    X, Y = [], []
    n_ent = n_sep = 0
    trials = 0

    while (n_ent < n or n_sep < n) and trials < max_trials:
        trials += 1

        # ---- sample graph ----
        p_sample = np.random.uniform(p_min, p_max)
        A = random_weighted_graph_qutrit(p=p_sample)

        if np.sum(A) == 0:
            continue

        alpha0 = alpha_positivity(A)
        if alpha0 >= 1.0:
            continue

        alpha_ppt = alpha_ppt_threshold(A)

        # ---------- ENTANGLED ----------
        if n_ent < n and alpha0 < alpha_ppt:
            alphas_ent = stratified_alphas(
                alpha0, alpha_ppt, k_ent, eps_frac
            )

            for alpha in alphas_ent:
                if n_ent >= n:
                    break
                if classify_state_from_graph(A, alpha) == "ENT":
                    rho = density_matrix_from_graph(A, alpha)
                    if rho is None:
                        continue

                    for _ in range(k_lu):
                        rho_aug = apply_local_unitary(rho)
                        X.append(gell_mann_features(rho_aug))
                        Y.append(1)
                        n_ent += 1
                        if n_ent >= n:
                            break

        # ---------- SEPARABLE ----------
        alpha_min = max(alpha0, alpha_ppt)
        if n_sep < n and alpha_min < 1.0:
            alphas_sep = stratified_alphas(
                alpha_min, 1.0, k_sep, eps_frac
            )

            for alpha in alphas_sep:
                if n_sep >= n:
                    break
                if classify_state_from_graph(A, alpha) == "SEP":
                    rho = density_matrix_from_graph(A, alpha)
                    if rho is None:
                        continue

                    for _ in range(k_lu):
                        rho_aug = apply_local_unitary(rho)
                        X.append(gell_mann_features(rho_aug))
                        Y.append(0)
                        n_sep += 1
                        if n_sep >= n:
                            break

        if (n_ent + n_sep) % 2000 == 0 and (n_ent + n_sep) > 0:
            print(
                f"ENT: {n_ent}/{n}, "
                f"SEP: {n_sep}/{n}, "
                f"graphs tried: {trials}"
            )

    if n_ent < n or n_sep < n:
        raise RuntimeError(
            f"Could not build balanced dataset. "
            f"ENT={n_ent}, SEP={n_sep}, trials={trials}"
        )

    return np.array(X), np.array(Y)

In [ ]:
import os
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping



In [ ]:
X, y = build_dataset(
    n=50000,
    p_min=0.1,
    p_max=0.9,
    k_ent=3,
    k_sep=3,
    k_lu=2,
    eps_frac=0.1
)

X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp
)

print("Train:", X_train.shape)
print("Val:", X_val.shape)
print("Test:", X_test.shape)



ENT: 3876/50000, SEP: 2124/50000, graphs tried: 824
ENT: 7782/50000, SEP: 4218/50000, graphs tried: 1664
ENT: 11688/50000, SEP: 6312/50000, graphs tried: 2509
ENT: 15612/50000, SEP: 8388/50000, graphs tried: 3345
ENT: 23274/50000, SEP: 12726/50000, graphs tried: 5006
ENT: 27156/50000, SEP: 14844/50000, graphs tried: 5834
ENT: 30942/50000, SEP: 17058/50000, graphs tried: 6629
ENT: 34782/50000, SEP: 19218/50000, graphs tried: 7449
ENT: 38682/50000, SEP: 21318/50000, graphs tried: 8286
ENT: 42618/50000, SEP: 23382/50000, graphs tried: 9110
ENT: 46464/50000, SEP: 25536/50000, graphs tried: 9961
ENT: 50000/50000, SEP: 30000/50000, graphs tried: 11672
ENT: 50000/50000, SEP: 36000/50000, graphs tried: 14008
ENT: 50000/50000, SEP: 36000/50000, graphs tried: 14010
ENT: 50000/50000, SEP: 36000/50000, graphs tried: 14011
ENT: 50000/50000, SEP: 42000/50000, graphs tried: 16434
ENT: 50000/50000, SEP: 48000/50000, graphs tried: 18750
ENT: 50000/50000, SEP: 50000/50000, graphs tried: 19531
Train: (64

In [ ]:
def focal_loss(gamma=2.0, alpha=0.5):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        return -alpha * tf.reduce_mean((1 - pt)**gamma * tf.math.log(pt))
    return loss

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

def build_mlp(input_dim, seed):
    tf.keras.utils.set_random_seed(seed)

    model = Sequential([
        Dense(128, activation='relu',
              kernel_regularizer=l2(5e-5),
              input_shape=(input_dim,)),
        Dropout(0.1),

        Dense(64, activation='relu',
              kernel_regularizer=l2(5e-5)),
        Dropout(0.1),

        Dense(1, activation='sigmoid')
    ])
    return model


def train_single_mlp(X_train, y_train, X_val, y_val, seed):
    model = build_mlp(X_train.shape[1], seed)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
        loss=focal_loss(gamma=2.0),
        metrics=['accuracy']
    )

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=200,
        batch_size=128,
        callbacks=[early_stop],
        verbose=1
    )

    return model

N_ENSEMBLE = 5
SEEDS = [10, 20, 30, 40, 50]

models = []

for i, seed in enumerate(SEEDS):
    print(f"Training model {i+1}/{N_ENSEMBLE} (seed={seed})")
    model = train_single_mlp(
        X_train_scaled, y_train,
        X_val_scaled, y_val,
        seed
    )
    models.append(model)




Training model 1/5 (seed=10)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5864 - loss: 0.0978 - val_accuracy: 0.8153 - val_loss: 0.0632
Epoch 2/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7990 - loss: 0.0610 - val_accuracy: 0.8618 - val_loss: 0.0463
Epoch 3/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8526 - loss: 0.0470 - val_accuracy: 0.8783 - val_loss: 0.0395
Epoch 4/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8762 - loss: 0.0402 - val_accuracy: 0.8879 - val_loss: 0.0360
Epoch 5/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8890 - loss: 0.0363 - val_accuracy: 0.8903 - val_loss: 0.0344
Epoch 6/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8973 - loss: 0.0338 - val_accuracy: 0.8957 - val_loss: 0.0330
Epoch 7/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9044 - loss: 0.0317 - val_accuracy: 0.8958 - val_loss: 0.0324
Epoch 8/200
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9113 - loss: 0.0299 - val_accu

In [ ]:
def ensemble_predict(models, X):
    probs = np.array([
        model.predict(X).ravel()
        for model in models
    ])
    return probs.mean(axis=0)


In [ ]:
from sklearn.metrics import accuracy_score

val_probs = ensemble_predict(models, X_val_scaled)

thresholds = np.linspace(0.2, 0.8, 301)
best_t, best_acc = 0.5, 0.0

for t in thresholds:
    acc = accuracy_score(y_val, (val_probs >= t).astype(int))
    if acc > best_acc:
        best_acc = acc
        best_t = t

print(f"Optimal threshold (val): {best_t:.3f}")
print(f"Validation accuracy: {best_acc:.4f}")


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Optimal threshold (val): 0.462
Validation accuracy: 0.9242


In [ ]:
test_probs = ensemble_predict(models, X_test_scaled)
test_preds = (test_probs >= best_t).astype(int)

test_acc = accuracy_score(y_test, test_preds)
test_auc = roc_auc_score(y_test, test_probs)

print(f"Ensemble Test Accuracy: {test_acc:.4f}")
print(f"Ensemble Test ROC-AUC:  {test_auc:.4f}")

print(classification_report(
    y_test,
    test_preds,
    target_names=["SEP", "ENT"]
))

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Ensemble Test Accuracy: 0.9221
Ensemble Test ROC-AUC:  0.9835
              precision    recall  f1-score   support

         SEP       0.92      0.93      0.92     10000
         ENT       0.93      0.92      0.92     10000

    accuracy                           0.92     20000
   macro avg       0.92      0.92      0.92     20000
weighted avg       0.92      0.92      0.92     20000



In [ ]:
indiv_probs = np.array([
    model.predict(X_test_scaled).ravel()
    for model in models
])

print("Mean disagreement:",
      np.mean(np.std(indiv_probs, axis=0)))


625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Mean disagreement: 0.045317367
